In [7]:
#%% Import general libraries
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from tezcatli_scripts import load_data, utils, pre_process , write_to_database as w2d #, get_ts_features , fit_models,
from tezcatli_scripts.fit_models import  Darts, Orbit

#import random
from random import getrandbits, seed
import pickle

#For reading options from command line
import sys, getopt

import warnings
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20,6]
pd.set_option('display.max_columns',12)
pd.set_option('display.width', 1000)

In [8]:
from orbit.models import DLT

In [9]:
#%% Setup Run ###
run_config = utils.read_params_in_from_json('run_config.json')
group_key = run_config['dimensions'].split('-')
#seed(2309)
run_id = getrandbits(32)
run_date = dt.datetime.today().date()
run_datascientist = run_config['data_scientist']
run_scope = run_config['scope']
run_response = run_config['response']
run_timegrain = run_config['timegrain']
run_dimensions = run_config['dimensions']
run_type = run_config['type']
holdout_horizon = run_config['train_horizon']
forecast_horizon = run_config['forecast_horizon']

test_run = True

write_file = 'yes'

In [11]:
#%% Models to run ###
#%% Models to run ###
model_frames = ['Orbit']
models = [DLT()]
models_names = ['Orbit']
orbit_models = [DLT()]
orbit_models_names = ['DampedLinearTrend']

In [12]:
run_mofcst = dt.datetime(run_config['current_year'],run_config['current_month'],1)
train_date = run_mofcst + relativedelta(months=-holdout_horizon)
order_min_date = dt.datetime(2014,4, 1).date()
order_max_date = run_mofcst + relativedelta(months=-1)

In [13]:
import dask.dataframe as dd
from dask.distributed import Client

In [14]:
client = Client()
client

C:\Users\PabloT\AppData\Local\anaconda3\envs\stan_env\lib\site-packages\distributed\node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60617 instead
  f"Port {expected} is already in use.\n"


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:60617/status,
Dashboard: http://127.0.0.1:60617/status,Workers: 4
Total threads: 16,Total memory: 31.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:60620,Workers: 4
Dashboard: http://127.0.0.1:60617/status,Total threads: 16
Started: Just now,Total memory: 31.21 GiB
Comm: tcp://127.0.0.1:60656,Total threads: 4
Dashboard: http://127.0.0.1:60657/status,Memory: 7.80 GiB
Nanny: tcp://127.0.0.1:60625,


In [15]:
data = dd.read_parquet('data/tezcatli_orders_data.parquet')

In [16]:
top_fcst_grps = ["Pr Plk","Pr HLD","Pr Pnl","C+ St Plk","Int 1/2 Inch","C+ St NT3","Int 1/4 Inch","C+ St Pnl","Pr CemPre","Pr Soff 12'","C+ St HLD","Pr Soff 8'"]
data = data[data['forecast_group'].isin(top_fcst_grps)]

In [18]:
data['forecast_group'].unique().compute()

0        C+ St Plk
1        C+ St NT3
2        C+ St Pnl
3           Pr Plk
4        C+ St HLD
5           Pr HLD
6           Pr Pnl
7      Pr Soff 12'
8       Pr Soff 8'
9     Int 1/2 Inch
10    Int 1/4 Inch
11       Pr CemPre
Name: forecast_group, dtype: object